In [1]:
import pandas as pd
import numpy as np
import vcf

In [20]:
class Pileup_line:
    def __init__(self, line): 
        split_line = line.split("\t")
        self.sequence = split_line[0]
        self.position = split_line[1]
        self.ref_base = split_line[2]
        self.read_count = split_line[3]
        self.read_results = split_line[4]
        self.base_quality = split_line[5]
        self.mapping_quality = split_line[6]
        
    # Pomocna funkcija za racunanje kvaliteta baze iz odgovarajuceg ascii karaktera
    # Phred quality score: https://en.wikipedia.org/wiki/Phred_quality_score
    def calc_base_quality(ascii_char):
        return 1-10**(-ord(ascii_char)/10)
    
    # Ista funkcija kao gore, samo sto se prosledi redni broj baze/read-a
    def calc_base_quality(self, base_num):
        # base_num < self.read_count !!!!
        return 1-10**(-ord(self.base_quality[base_num])/10)
    
    # Pomocna funkcija za racunanje broja pojavljivanja karaktera u stringu
    # Ovo sam mislila za prebrojavanje npr baza koje se poklapaju sa referentnom tj. broj . u read_results
    def num_of_occurence(self, char):
        return self.read_results.count(char)
    
    

In [26]:
with open('/sbgenomics/project-files/merged-normal_pileup.pileup', 'r') as pileup_file:
    i = 0
    for line in pileup_file:
        a = Pileup_line(line)
        print(line)
        print(a.num_of_occurence(','))
        i+=1
        if i==30: 
            break

21	9483248	C	1	^<.	A	<

0
21	9483249	T	1	.	B	<

0
21	9483250	T	1	.	>	<

0
21	9483251	T	1	.	>	<

0
21	9483252	T	1	.	B	<

0
21	9483253	C	1	.	F	<

0
21	9483254	A	1	.	A	<

0
21	9483255	A	1	.	F	<

0
21	9483256	A	2	.^<.	DA	<<

0
21	9483257	C	2	..	IA	<<

0
21	9483258	A	2	..	D@	<<

0
21	9483259	G	2	..	JD	<<

0
21	9483260	T	2	..	?=	<<

0
21	9483261	A	2	..	@@	<<

0
21	9483262	T	2	..	@@	<<

0
21	9483263	C	2	..	JG	<<

0
21	9483264	T	2	..	DD	<<

0
21	9483265	A	2	..	??	<<

0
21	9483266	T	3	..^<.	@@>	<<<

0
21	9483267	G	3	...	HJ<	<<<

0
21	9483268	C	3	...	HG?	<<<

0
21	9483269	C	3	...	GF@	<<<

0
21	9483270	T	3	...	DE@	<<<

0
21	9483271	G	3	...	JIC	<<<

0
21	9483272	C	3	...	HIH	<<<

0
21	9483273	C	3	...	GFF	<<<

0
21	9483274	A	3	...	CC@	<<<

0
21	9483275	A	3	...	EEC	<<<

0
21	9483276	A	3	...	EEC	<<<

0
21	9483277	T	3	...	@@A	<<<

0
